In [ ]:
"""
Jeff Wiederkehr
Sample Project
Update Map Script

This script takes a CSV file uploads it to ArcGIS, maps the uploaded file, then creates separate
feature classes for data in the safety score field of 3 or less, 4-7 and 7 or more applying red,
yellow and green circles respectively
"""

import arcpy

#set field to allow up loaded data to be added to the map
arcpy.env.addOutputsToMap = True

# set workspace to current enviroment gdb
arcpy.env.workspace = r"C:\Users\wiede\Documents\ArcGIS\Projects\MyProjectTest\MyProjectTest.gdb"

# specify CSV file locattion hardcoded but can be easily adjusted to grab different CSV's
in_table = r"C:\Users\wiede\Desktop\CSV_Data\SampleData.csv"

# add CSV data to project as XYTable
arcpy.management.XYTableToPoint(in_table, r"C:\Users\wiede\Documents\ArcGIS\Projects\MyProjectTest\MyProjectTest.gdb\SampleDataXY", "Longitude", "Latitude", None, "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision")

# set current project
cur_project = arcpy.mp.ArcGISProject("CURRENT")
# set current project
cur_project = arcpy.mp.ArcGISProject("CURRENT")

# create dictionary of colors and their RGB values, yellow is a mustard yellow for map visibility
color_RGB_dict = {"red": [255, 0, 0, 60], "yellow": [255, 173, 1, 60], "green": [0, 128, 0, 60]}


def create_new_features():
    """
    Add new feature layers to the cur project map for red, green and yellow colors. Red corresponds with a Safety score
    3 or lower. Yellow is 7-4, Green is 7 or greater
    """
    red = arcpy.SelectLayerByAttribute_management("SampleDataXY", "NEW_SELECTION", "SafetyScore <= 3")
    arcpy.CopyFeatures_management(red, 'red')
    green = arcpy.SelectLayerByAttribute_management("SampleDataXY", "NEW_SELECTION", "SafetyScore >= 7")
    arcpy.CopyFeatures_management(green, 'green')
    yellow = arcpy.SelectLayerByAttribute_management("SampleDataXY", "NEW_SELECTION",
                                                     "SafetyScore < 7 AND SafetyScore > 3")
    arcpy.CopyFeatures_management(yellow, 'yellow')


def update_symbol(color_feature):
    """
    Updates the symbol color for of each layer created in create_new_features. Symbol color is set to a new RGB value
    from color_RGB_dict based on the features name
    :param color_feature: Symbol object, updated to change color
    """
    rgb = color_RGB_dict[color_feature.name]
    sym = color_feature.symbology
    sym.renderer.symbol.applySymbolFromGallery("Circle 1")
    sym.renderer.symbol.color = {'RGB': rgb}
    color_feature.symbology = sym


create_new_features()
# Hard codded due to only one map in project if multiple maps would iterate and select by name
map_list = cur_project.listMaps()[0]
feature_list = map_list.listLayers()
for feature in feature_list:
    if feature.name in color_RGB_dict:
        update_symbol(feature)
    if feature.name == "SampleDataXY":
        feature.visible = False

cur_project.save()

In [ ]:
"""
Jeff Wiederkehr
Sample Project
Upload Map Script

This script takes the newly updated map and uploads it to ArcGIS Online
"""
import arcpy
import os

# Set name for file and create sd and sddraft files
cwd = os.path.abspath(os.getcwd())
map_name = "Jeff__Wiederkehr_Map"
sd_name = fr"\{map_name}.sd"
sd_draft_name = fr"\{map_name}.sddraft"
sd_file = fr"{cwd}{sd_name}"
sd_draft_file = fr"{cwd}{sd_draft_name}"

# Select current project
aprx = arcpy.mp.ArcGISProject("CURRENT")

# Select map to upload
cur_map = aprx.listMaps()[0]

# converts map into sddraft
arcpy.mp.CreateWebLayerSDDraft(cur_map, sd_draft_file, map_name, 'MY_HOSTED_SERVICES', 'FEATURE_ACCESS')

# converts map to sd file
arcpy.StageService_server(sd_draft_file, sd_file)

# uploads sd file to online portal
arcpy.UploadServiceDefinition_server( sd_file, 'MY_HOSTED_SERVICES')